In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import cv2
import torch.optim as optim
import torch
from torch.utils.data import Dataset, DataLoader
from cv2 import dnn_superres
import torch.nn as nn
import torch.nn.functional as F


In [254]:
images = glob.glob('*.jpg')
data = pd.DataFrame(images, columns=['src'])

In [255]:
data['brand'] = data['src'].apply(lambda x : x.split('_')[0].split('/')[-1])

In [256]:
d1 = data[data['brand']=='BMW'][:50]
d2 = data[data['brand']=='Audi'][:50]
d3 = data[data['brand']=='Nissan'][:50] 
d4 = data[data['brand']=='Mercedes-Benz'][:50] 
d5 = data[data['brand']=='Honda'][:50] 

frames = [d1, d2, d3, d4, d5]

data = pd.concat(frames)

In [257]:
data = data.reset_index(drop=True)

In [258]:
labels_map = {
    "BMW": 0,
    "Audi": 1,
    "Mercedes-Benz": 2,
    "Nissan": 3,
    "Honda": 4,
}

In [131]:
def super_resolution(images):
    sr = dnn_superres.DnnSuperResImpl_create()
    new_images_list = []
    path = "FSRCNN_x4.pb"
    sr.readModel(path)
    sr.setModel("fsrcnn", 4)
    for image in images:
        new_image = cv2.resize(image, dsize = (256, 256), interpolation=cv2.INTER_AREA)
        new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
        result = sr.upsample(new_image)
        new_images_list.append(result)
    return new_images_list

In [260]:
label_list = []
for i in range(len(data)):
    label_list.append(labels_map[data["brand"][i]])
data["label"] = label_list
data

,src,brand,label
0,BMW_2-Series_2014_32_17_240_20_4_69_55_174_23_...,BMW,0
1,BMW_2-Series_2014_32_17_240_20_4_69_55_174_23_...,BMW,0
2,BMW_2-Series_2014_32_17_240_20_4_69_55_174_23_...,BMW,0
3,BMW_2-Series_2014_32_17_240_20_4_69_55_174_23_...,BMW,0
4,BMW_2-Series_2014_32_17_240_20_4_69_55_174_23_...,BMW,0
...,...,...,...
245,Honda_Accord_2011_25_17_190_24_4_72_56_191_22_...,Honda,4
246,Honda_Accord_2011_25_17_190_24_4_72_56_191_22_...,Honda,4
247,Honda_Accord_2011_25_17_190_24_4_72_56_191_22_...,Honda,4
248,Honda_Accord_2011_25_17_190_24_4_72_56_191_22_...,Honda,4


In [261]:
X= []
y= []
dict_pairs = {}
for i in range(len(data)):
    src = data.loc[i,'src']
    src = cv2.imread(src, cv2.IMREAD_COLOR)
    X.append(src)
    y.append(data.loc[i,'label'])
    

In [263]:
new_X = super_resolution(X)

In [265]:
class CustomImageDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
        
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx].reshape(-1,32,32)/255, np.array(self.labels[idx])

In [266]:
itera_dataset = CustomImageDataset(new_X, y)

In [267]:
train_loader = DataLoader(itera_dataset, batch_size=32, shuffle=False)

In [268]:
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    print('Class labels of 10 examples:', labels[:10])
    break

Image batch dimensions: torch.Size([32, 1024, 32, 32])
Image label dimensions: torch.Size([32])
Class labels of 10 examples: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [269]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = torch.nn.Sequential(
                torch.nn.Conv2d(1024, 256, kernel_size=3, stride=2, padding=1),
                torch.nn.BatchNorm2d(256),
                torch.nn.LeakyReLU(0.1, inplace=True),
                #
                torch.nn.Conv2d(256, 2048, kernel_size=3, stride=1, padding=1),
                torch.nn.BatchNorm2d(2048),
                torch.nn.LeakyReLU(0.1, inplace=True),
                #
                torch.nn.Conv2d(2048, 64, kernel_size=3, stride=2, padding=1),
                torch.nn.BatchNorm2d(64),
                torch.nn.LeakyReLU(0.1, inplace=True),
                #
                torch.nn.Flatten(),
                torch.nn.Linear(4096, 128),
                torch.nn.BatchNorm1d(128),
                torch.nn.LeakyReLU(0.1, inplace=True),
                torch.nn.Dropout(0.5),
                #
                torch.nn.Linear(128, 5),
                )

    def forward(self, x):
        logits = self.layers(x)
        return logits


net = Net()

In [270]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [271]:
net = net.float()
for epoch in range(50):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        


        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

Finished Training


In [274]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in train_loader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images.float())
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network: %d %%' % (
    100 * correct / total))

Accuracy of the network: 86 %
